In [1]:
import pandas as pd
import gzip
from protein_clip import get_model, embed_sequence, batch_embed_sequence
from tqdm import tqdm

In [2]:
with gzip.open("../data/uniprot_sprot.paruqet.gz") as f:
    df = pd.read_parquet(f)
df
del f

In [15]:
df = df[df["sequence"].apply(len) < 3_000].copy()

In [4]:
# I want to run all the sequences through the esm model. Which size?
layer = 6
model, alphabet = get_model(layer)
model

ESM2(
  (embed_tokens): Embedding(33, 320, padding_idx=1)
  (layers): ModuleList(
    (0-5): 6 x TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=320, out_features=320, bias=True)
        (v_proj): Linear(in_features=320, out_features=320, bias=True)
        (q_proj): Linear(in_features=320, out_features=320, bias=True)
        (out_proj): Linear(in_features=320, out_features=320, bias=True)
        (rot_emb): RotaryEmbedding()
      )
      (self_attn_layer_norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=320, out_features=1280, bias=True)
      (fc2): Linear(in_features=1280, out_features=320, bias=True)
      (final_layer_norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
    )
  )
  (contact_head): ContactPredictionHead(
    (regression): Linear(in_features=120, out_features=1, bias=True)
    (activation): Sigmoid()
  )
  (emb_layer_norm_after): LayerNorm((320,), eps=1e-05, elementwis

In [49]:
import numpy as np
df["embedding"] = df["sequence"].apply(lambda x: np.zeros(0))
df

,accession,name,description,sequence,embedding
0,Q6GZX4,001R_FRG3G,Putative transcription factor 001R OS=Frog vir...,MAFSAEDVLKEYDRRRRMEALLLSLYYPNDRKLLDYKEWSPPRVQV...,[]
1,Q6GZX3,002L_FRG3G,Uncharacterized protein 002L OS=Frog virus 3 (...,MSIIGATRLQNDKSDTYSAGPCYAGGCSAFTPRGTCGKDWDLGEQT...,[]
2,Q197F8,002R_IIV3,Uncharacterized protein 002R OS=Invertebrate i...,MASNTVSAQGGSNRPVRDFSNIQDVAQFLLFDPIWNEQPGSIVPWK...,[]
3,Q197F7,003L_IIV3,Uncharacterized protein 003L OS=Invertebrate i...,MYQAINPCPQSWYGSPQLEREIVCKMSGAPHYPNYYPVHPNALGGA...,[]
4,Q6GZX2,003R_FRG3G,Uncharacterized protein 3R OS=Frog virus 3 (is...,MARPLLGKTSSVRRRLESLSACSIFFFLRKFCQKMASLVFLNSPVY...,[]
...,...,...,...,...,...
570152,Q6UY62,Z_SABVB,RING finger protein Z OS=Sabia mammarenavirus ...,MGNSKSKSKLSANQYEQQTVNSTKQVAILKRQAEPSLYGRHNCRCC...,[]
570153,P08105,Z_SHEEP,Putative uncharacterized protein Z OS=Ovis ari...,MSSSLEITSFYSFIWTPHIGPLLFGIGLWFSMFKEPSHFCPCQHPH...,[]
570154,Q88470,Z_TACVF,RING finger protein Z OS=Tacaribe virus (strai...,MGNCNRTQKPSSSSNNLEKPPQAAEFRRTAEPSLYGRYNCKCCWFA...,[]
570155,A9JR22,Z_TAMVU,RING finger protein Z OS=Tamiami mammarenaviru...,MGLRYSKEVRDRHGDKDPEGRIPITQTMPQTLYGRYNCKSCWFANK...,[]


In [54]:
checkpoint_every = 1_000
not_computed_yet = df[df["embedding"].apply(len) == 0]
j = 0
for i, row in tqdm(not_computed_yet.iterrows(), total=len(not_computed_yet)):
    embed = embed_sequence(row["sequence"], model, alphabet, layer)
    df.at[i, "embedding"] = embed
    
    # save to disk every once and a while
    if j > checkpoint_every:
        df.to_parquet(f"uniprot-esm2-layers{layer}.parquet")
        j = 0
    j += 1

  1%|          | 3983/562736 [07:14<16:56:00,  9.17it/s] 


KeyboardInterrupt: 